<img src= 'covid.gif' width=30% />

# Import packages

In [1]:
import re
from textblob import TextBlob
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))
import random
import string
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import json
import glob
import sys
import spacy
import scispacy  # https://allenai.github.io/scispacy/
spacy.prefer_gpu()
from spacy import displacy

# nlp = spacy.load("en_core_web_sm")
sys.path.insert(0, "../")


from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Load clean data (https://www.kaggle.com/xhlulu/cord-19-eda-parse-json-and-generate-clean-csv)

In [2]:
df=pd.read_csv('D:/GraphDB/corona/NLP/2020-03-13/CDC/COVID19_Excel_20March2020.csv')

In [3]:
df.head(10)

,Author,Title,Year,Abstract,Journal/Publisher,Volume,Issue,Pages,Accession Number,DOI,URL,Name of Database,Database Provider,Language,Keywords
0,"Zhang, YHL, D. J.; Xiao, M. F.; Wang, J. C.; W...",[2019 novel coronavirus infection in a three-m...,2020,NaN,Zhonghua Erke Zazhi,58.0,3,182-184,32135587,https://dx.doi.org/10.3760/cma.j.issn.0578-131...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,*Betacoronavirus; *Coronavirus Infections/di [...
1,"Shi, HZM, P.; Gao, F. Y.; Chen, G. L.; Yu, Y. ...","[2019 novel coronavirus, angiotensin convertin...",2020,NaN,Chung-Hua Hsin Hsueh Kuan Ping Tsa Chih [Chine...,48.0,0,E010,32166940,https://dx.doi.org/10.3760/cma.j.cn112148-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
2,"Tian, HY",[2019-nCoV: new challenges from coronavirus],2020,The outbreak of pneumonia caused by the novel ...,Chung-Hua Yu Fang i Hsueh Tsa Chih [Chinese Jo...,54.0,0,E001,32023682,https://dx.doi.org/10.3760/cma.j.issn.0253-962...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
3,"Zhang, YHL, D. J.; Xiao, M. F.; Wang, J. C.; W...",[2019-novel coronavirus infection in a three-m...,2020,NaN,Zhonghua Erke Zazhi,58.0,0,E006,32043842,https://dx.doi.org/10.3760/cma.j.issn.0578-131...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
4,"Yao, XHL, T. Y.; He, Z. C.; Ping, Y. F.; Liu, ...",[A pathological report of three COVID-19 cases...,2020,<b>Objective:</b> To investigate the pathologi...,Chung-Hua Ping Li Hsueh Tsa Chih - Chinese Jou...,49.0,0,E009,32172546,https://dx.doi.org/10.3760/cma.j.cn112151-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
5,"Yang, JMM, X.; Xue, F.; Zhang, Y.; Zhang, C.",[ACE2 in the context of 2019-nCoV infection: f...,2020,NaN,Chung-Hua Hsin Hsueh Kuan Ping Tsa Chih [Chine...,48.0,0,E012,32189489,https://dx.doi.org/10.3760/cma.j.cn112148-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
6,"Chen, CZ, X. R.; Ju, Z. Y.; He, W. F.",[Advances in the research of cytokine storm me...,2020,Corona Virus Disease 2019 (COVID-19) has serio...,Zhonghua Shao Shang Za Zhi,36.0,0,E005,32114747,https://dx.doi.org/10.3760/cma.j.cn501120-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
7,"Wang, JW, B. J.; Yang, J. C.; Wang, M. Y.; Che...",[Advances in the research of mechanism of pulm...,2020,The Corona Virus Disease 2019 (COVID-19) outbr...,Zhonghua Shao Shang Za Zhi,36.0,0,E006,32174095,https://dx.doi.org/10.3760/cma.j.cn501120-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
8,"Gao, WJL, L. M.",[Advances on presymptomatic or asymptomatic ca...,2020,COVID-19 is rapidly spreading. Patients in inc...,Chung-Hua Liu Hsing Ping Hsueh Tsa Chih Chines...,41.0,0,485-488,32141279,https://dx.doi.org/10.3760/cma.j.cn112338-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
9,Anonymous,[Airway management of COVID-19 patients with s...,2020,Patients with severe and critical COVID-19 wil...,Zhonghua Er Bi Yan Hou Tou Jing Wai Ke Za Zhi ...,55.0,4,E001,32100976,https://dx.doi.org/10.3760/cma.j.issn.1673-086...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN


In [4]:
df = df[df['Abstract'].notna()]

In [5]:
# df2=df.iloc[[2,3],:]
# surveys_df.loc[[0, 10], :]

In [6]:
df.head()

,Author,Title,Year,Abstract,Journal/Publisher,Volume,Issue,Pages,Accession Number,DOI,URL,Name of Database,Database Provider,Language,Keywords
2,"Tian, HY",[2019-nCoV: new challenges from coronavirus],2020,The outbreak of pneumonia caused by the novel ...,Chung-Hua Yu Fang i Hsueh Tsa Chih [Chinese Jo...,54.0,0,E001,32023682,https://dx.doi.org/10.3760/cma.j.issn.0253-962...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
4,"Yao, XHL, T. Y.; He, Z. C.; Ping, Y. F.; Liu, ...",[A pathological report of three COVID-19 cases...,2020,<b>Objective:</b> To investigate the pathologi...,Chung-Hua Ping Li Hsueh Tsa Chih - Chinese Jou...,49.0,0,E009,32172546,https://dx.doi.org/10.3760/cma.j.cn112151-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
6,"Chen, CZ, X. R.; Ju, Z. Y.; He, W. F.",[Advances in the research of cytokine storm me...,2020,Corona Virus Disease 2019 (COVID-19) has serio...,Zhonghua Shao Shang Za Zhi,36.0,0,E005,32114747,https://dx.doi.org/10.3760/cma.j.cn501120-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
7,"Wang, JW, B. J.; Yang, J. C.; Wang, M. Y.; Che...",[Advances in the research of mechanism of pulm...,2020,The Corona Virus Disease 2019 (COVID-19) outbr...,Zhonghua Shao Shang Za Zhi,36.0,0,E006,32174095,https://dx.doi.org/10.3760/cma.j.cn501120-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN
8,"Gao, WJL, L. M.",[Advances on presymptomatic or asymptomatic ca...,2020,COVID-19 is rapidly spreading. Patients in inc...,Chung-Hua Liu Hsing Ping Hsueh Tsa Chih Chines...,41.0,0,485-488,32141279,https://dx.doi.org/10.3760/cma.j.cn112338-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN


In [7]:
# noncomm_df=pd.read_csv('./clean_noncomm_use.csv')

In [8]:
# pmc_df=pd.read_csv('./clean_pmc.csv')

## NLP pre-process

### Remove Punctuation

In [9]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

In [10]:
df['no_punkt'] = df['Abstract'].progress_apply(lambda x: remove_punct(x))

100%|████████████████████████████████████████████████████████████████████████████| 1918/1918 [00:00<00:00, 8523.90it/s]


### Tokenize

In [11]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

In [12]:
df['token'] = df['Abstract'].progress_apply(lambda x: tokenize(x.lower()))
df.dtypes
# df.head()

100%|███████████████████████████████████████████████████████████████████████████| 1918/1918 [00:00<00:00, 12216.72it/s]


Author                object
Title                 object
Year                  object
Abstract              object
Journal/Publisher     object
Volume               float64
Issue                 object
Pages                 object
Accession Number      object
DOI                   object
URL                   object
Name of Database      object
Database Provider     object
Language              object
Keywords              object
no_punkt              object
token                 object
dtype: object

### Remove stopwords

In [13]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

df['no_stop'] = df['token'].progress_apply(lambda x: remove_stopwords(x))

df.head()

100%|████████████████████████████████████████████████████████████████████████████| 1918/1918 [00:00<00:00, 2946.17it/s]


,Author,Title,Year,Abstract,Journal/Publisher,Volume,Issue,Pages,Accession Number,DOI,URL,Name of Database,Database Provider,Language,Keywords,no_punkt,token,no_stop
2,"Tian, HY",[2019-nCoV: new challenges from coronavirus],2020,The outbreak of pneumonia caused by the novel ...,Chung-Hua Yu Fang i Hsueh Tsa Chih [Chinese Jo...,54.0,0,E001,32023682,https://dx.doi.org/10.3760/cma.j.issn.0253-962...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN,The outbreak of pneumonia caused by the novel ...,"[the, outbreak, of, pneumonia, caused, by, the...","[outbreak, pneumonia, caused, novel, coronavir..."
4,"Yao, XHL, T. Y.; He, Z. C.; Ping, Y. F.; Liu, ...",[A pathological report of three COVID-19 cases...,2020,<b>Objective:</b> To investigate the pathologi...,Chung-Hua Ping Li Hsueh Tsa Chih - Chinese Jou...,49.0,0,E009,32172546,https://dx.doi.org/10.3760/cma.j.cn112151-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN,bObjectiveb To investigate the pathological ch...,"[, b, objective, b, to, investigate, the, path...","[, b, objective, b, investigate, pathological,..."
6,"Chen, CZ, X. R.; Ju, Z. Y.; He, W. F.",[Advances in the research of cytokine storm me...,2020,Corona Virus Disease 2019 (COVID-19) has serio...,Zhonghua Shao Shang Za Zhi,36.0,0,E005,32114747,https://dx.doi.org/10.3760/cma.j.cn501120-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN,Corona Virus Disease 2019 COVID19 has seriousl...,"[corona, virus, disease, 2019, covid, 19, has,...","[corona, virus, disease, 2019, covid, 19, seri..."
7,"Wang, JW, B. J.; Yang, J. C.; Wang, M. Y.; Che...",[Advances in the research of mechanism of pulm...,2020,The Corona Virus Disease 2019 (COVID-19) outbr...,Zhonghua Shao Shang Za Zhi,36.0,0,E006,32174095,https://dx.doi.org/10.3760/cma.j.cn501120-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN,The Corona Virus Disease 2019 COVID19 outbroke...,"[the, corona, virus, disease, 2019, covid, 19,...","[corona, virus, disease, 2019, covid, 19, outb..."
8,"Gao, WJL, L. M.",[Advances on presymptomatic or asymptomatic ca...,2020,COVID-19 is rapidly spreading. Patients in inc...,Chung-Hua Liu Hsing Ping Hsueh Tsa Chih Chines...,41.0,0,485-488,32141279,https://dx.doi.org/10.3760/cma.j.cn112338-2020...,http://ovidsp.ovid.com/ovidweb.cgi?T=JS&CSC=Y&...,MEDLINE,Ovid Technologies,Chinese,NaN,COVID19 is rapidly spreading Patients in incub...,"[covid, 19, is, rapidly, spreading, patients, ...","[covid, 19, rapidly, spreading, patients, incu..."


In [83]:
print(df.Title[4])

[A pathological report of three COVID-19 cases by minimally invasive autopsies]


## Visualize Bio-Medical entities 

In [15]:
import random 

def get_entity_options(random_colors=False):
    """
    generating color options for visualizing the named entities
    """
    def color_generator(number_of_colors):
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                 for i in range(number_of_colors)]
        return color

    entities = ["GGP", "SO", "TAXON", "CHEBI", "GO", "CL", 
                "DNA", "CELL_TYPE", "CELL_LINE", "RNA", "PROTEIN",
                "DISEASE", "CHEMICAL",
                "CANCER", "ORGAN", "TISSUE", "ORGANISM", "CELL", 
                "AMINO_ACID", "GENE_OR_GENE_PRODUCT", "SIMPLE_CHEMICAL", 
                "ANATOMICAL_SYSTEM", "IMMATERIAL_ANATOMICAL_ENTITY", "MULTI-TISSUE_STRUCTURE",
                "DEVELOPING_ANATOMICAL_STRUCTURE", "ORGANISM_SUBDIVISION", "CELLULAR_COMPONENT"]
    
    colors = {"ENT":"#82E0AA"}
    
    if random_colors:
        color = color_generator(len(entities))
        for i in range(len(entities)):
            colors[entities[i]] = color[i]
    else:
        entities_cat_1 = {"GGP":"#F9E79F", "SO":"#F7DC6F", "TAXON":"#F4D03F", "CHEBI":"#FAD7A0", "GO":"#F8C471", "CL":"#F5B041"}
        entities_cat_2 = {"DNA":"#82E0AA", "CELL_TYPE":"#AED6F1", "CELL_LINE":"#E8DAEF", "RNA":"#82E0AA", "PROTEIN":"#82E0AA"}
        entities_cat_3 = {"DISEASE":"#D7BDE2", "CHEMICAL":"#D2B4DE"}
        entities_cat_4 = {"CANCER":"#ABEBC6", "ORGAN":"#82E0AA", "TISSUE":"#A9DFBF", "ORGANISM":"#A2D9CE", "CELL":"#76D7C4", "AMINO_ACID":"#85C1E9", "GENE_OR_GENE_PRODUCT":"#AED6F1", "SIMPLE_CHEMICAL":"#76D7C4", "ANATOMICAL_SYSTEM":"#82E0AA", "IMMATERIAL_ANATOMICAL_ENTITY":"#A2D9CE", "MULTI-TISSUE_STRUCTURE":"#85C1E9", "DEVELOPING_ANATOMICAL_STRUCTURE":"#A9DFBF", "ORGANISM_SUBDIVISION":"#58D68D", "CELLULAR_COMPONENT":"#7FB3D5"}

        entities_cats = [entities_cat_1, entities_cat_2, entities_cat_3, entities_cat_4]
        for item in entities_cats:
            colors = {**colors, **item}
    
    options = {"ents": entities, "colors": colors}
    
    return options

In [90]:
from spacy import displacy
nlp = spacy.load("en_ner_bionlp13cg_md")
# nlp = spacy.load("en_ner_bc5cdr_md")
# doc = nlp(" ".join(df.Abstract[4]))
doc = nlp(df.Abstract[6])
displacy.render(doc, style="ent", options=get_entity_options(random_colors=True))
# displacy.render(doc, style="ent",options=options)

In [ ]:
# nlp = spacy.load("en_ner_bionlp13cg_md")
# # nlp = spacy.load("en_ner_bc5cdr_md")
# df['new_text']= df['Abstract'].progress_apply(lambda x:list(nlp(x).ents))
# df.head()

In [ ]:
# def spacy_entity(df):    
#     df2=[[w.text,w.label_] for w in df1.ents]
#     return df2

In [ ]:
# df1['new_text']= df1['Abstract'].progress_apply(spacy_entity)
# print(df1['new_text'])

In [ ]:
# doc = nlp.make_doc(query)
# #nlp.tagger(doc)
# ner(doc)
# for word in doc:
#     print(word.text, word.orth, word.lower, word.ent_type_)

# for word in doc:
#     if word.ent_type_:
#         print ('word -> {} and entity-> {}'.format(word.text,word.ent_type_))
#     #print(word.text, word.orth, word.lower, word.ent_type_)

In [129]:
from spacy import displacy
nlp = spacy.load("en_ner_bionlp13cg_md")
# nlp = spacy.load("en_ner_bc5cdr_md")
# doc = nlp(" ".join(df.Abstract[4]))

for index,row in df.iterrows():
    doc = nlp(df.Abstract[index])
#     print(doc)
    for word in doc:
        if word.ent_type_:
#             print ((df.Title,word.text,word.ent_type_))
            df['ner_text']=(str((df.Title,word.text,word.ent_type_)))
    print(df.ner_text)

2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                            

Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new chal

2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                            

Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new chal

2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                            

...

2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                            

2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                            

Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new chal

2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                            

2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                              ...                        
3367    (2            [2019-nCoV: new challenges from ...
3368    (2            [2019-nCoV: new challenges from ...
3369    (2            [2019-nCoV: new challenges from ...
3370    (2            [2019-nCoV: new challenges from ...
3372    (2            [2019-nCoV: new challenges from ...
Name: ner_text, Length: 1918, dtype: object
2       (2            [2019-nCoV: new challenges from ...
4       (2            [2019-nCoV: new challenges from ...
6       (2            [2019-nCoV: new challenges from ...
7       (2            [2019-nCoV: new challenges from ...
8       (2            [2019-nCoV: new challenges from ...
                            

In [112]:
df.ner_text[2]

"(2            [2019-nCoV: new challenges from coronavirus]\n4       [A pathological report of three COVID-19 cases...\n6       [Advances in the research of cytokine storm me...\n7       [Advances in the research of mechanism of pulm...\n8       [Advances on presymptomatic or asymptomatic ca...\n                              ...                        \n3367                             新型冠状病毒肺炎疫情防控期间医用 护目镜防雾技巧\n3368                 新型冠状病毒肺炎疫情防控期间武汉大型 综合医院内科普通门诊预检分诊及管理\n3369                         新型冠状病毒肺炎防疫中高效消毒剂的职业暴露风险和防护对策\n3370                  灾害脆弱性分析在非封闭式血液内科病房 应对新型冠状病毒传播中的应用效果\n3372                      首批抗击新型冠状病毒感染肺炎的临床 一线支援护士的心理状况调査\nName: Title, Length: 1918, dtype: object, '总结新型冠状病毒肺炎疫情防控期间武汉大型综合医院内科普通门诊预检分诊及管理经验，提出由经过专', 'GENE_OR_GENE_PRODUCT')"

In [128]:
from spacy import displacy
nlp = spacy.load("en_ner_bionlp13cg_md")
# nlp = spacy.load("en_ner_bc5cdr_md")
# doc = nlp(" ".join(df.Abstract[4]))
ind=11
doc = nlp(df.Abstract[ind])
for word in doc:
    if word.ent_type_:
#         entList.update(word.ent_type)
        print ((df.Title[ind],word.text,word.ent_type_))
        df['ner_text']=",".join((df.Title[ind],word.text,word.ent_type_))
    

('[Analysis of bronchoscope-guided tracheal intubation in 12 cases with COVID-19 under the personal protective equipment with positive pressure protective hood]', 'Endotracheal', 'MULTI-TISSUE_STRUCTURE')
('[Analysis of bronchoscope-guided tracheal intubation in 12 cases with COVID-19 under the personal protective equipment with positive pressure protective hood]', 'COVID-19', 'SIMPLE_CHEMICAL')
('[Analysis of bronchoscope-guided tracheal intubation in 12 cases with COVID-19 under the personal protective equipment with positive pressure protective hood]', 'endotracheal', 'PATHOLOGICAL_FORMATION')
('[Analysis of bronchoscope-guided tracheal intubation in 12 cases with COVID-19 under the personal protective equipment with positive pressure protective hood]', 'Guangzhou', 'ORGANISM')
('[Analysis of bronchoscope-guided tracheal intubation in 12 cases with COVID-19 under the personal protective equipment with positive pressure protective hood]', 'eighth', 'ORGANISM')
('[Analysis of bronchos

'[Advances on presymptomatic or asymptomatic carrier transmission of COVID-19],COVID-19,GENE_OR_GENE_PRODUCT'

In [58]:
print([(word, word.ent_type_) for word in doc if word.ent_type_])

[(coronavirus, 'ORGANISM'), (Wuhan, 'ORGANISM'), (Hubei, 'ORGANISM'), (province, 'ORGANISM'), (beta, 'GENE_OR_GENE_PRODUCT'), (genus, 'GENE_OR_GENE_PRODUCT'), (Coronavirus, 'GENE_OR_GENE_PRODUCT'), (SARS-CoV, 'ORGANISM'), (MERS-CoV, 'GENE_OR_GENE_PRODUCT'), (droplets, 'CELLULAR_COMPONENT'), (patients, 'ORGANISM'), (persons, 'ORGANISM'), (angiotensin-converting, 'GENE_OR_GENE_PRODUCT'), (enzyme, 'GENE_OR_GENE_PRODUCT'), (II, 'GENE_OR_GENE_PRODUCT'), (ACE2, 'GENE_OR_GENE_PRODUCT'), (SARS-CoV., 'GENE_OR_GENE_PRODUCT'), (coronavirus, 'ORGANISM')]


In [ ]:
nlp.max_length = 3300000

## Let’s now find out the most frequent named entities in our news corpus! For this, we will build out a data frame of all the named entities and their types using the following code.


In [ ]:

named_entities = []
# listToStr = ' '.join([str(elem) for elem in df.no_stop]) 

for row in tqdm(df.iterrows()):
    doc = nlp(df.no_stop_corpus) 
    for word in doc:        
        term = word.text 
        tag = word.ent_type_
        if tag:
            temp_entity_name = ' '.join([temp_entity_name, term]).strip()
            temp_named_entity = (temp_entity_name, tag)
        else:
            if temp_named_entity:
                named_entities.append(temp_named_entity)
                temp_entity_name = ''
                temp_named_entity = None

entity_frame = pd.DataFrame(named_entities, 
                            columns=['Entity Name', 'Entity Type'])


In [ ]:
print(named_entities)

In [ ]:

named_entities = []
for row in sentence:
    temp_entity_name = ''
    temp_named_entity = None
    sentence = nlp(sentence)
    for word in sentence:
        term = word.text 
        tag = word.ent_type_
        if tag:
            temp_entity_name = ' '.join([temp_entity_name, term]).strip()
            temp_named_entity = (temp_entity_name, tag)
        else:
            if temp_named_entity:
                named_entities.append(temp_named_entity)
                temp_entity_name = ''
                temp_named_entity = None

entity_frame = pd.DataFrame(named_entities, 
                            columns=['Entity Name', 'Entity Type'])

In [ ]:
# get the top named entities
top_entities = (entity_frame.groupby(by=['Entity Name', 'Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
top_entities.T.iloc[:,:15]

In [ ]:
top_entities = (entity_frame.groupby(by=['Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
# top_entities.T.iloc[:,:20]
top_entities.T.iloc[:,:30]

In [ ]:
import requests

def query_raw(text, url="https://bern.korea.ac.kr/plain"):
    return requests.post(url, data={'sample_text': text}).json()

if __name__ == '__main__':
    print(query_raw('To investigate the pathological characteristics and the clinical significance of novel coronavirus (2019-nCoV)-infected pneumonia (termed by WHO as corona virus disease 2019, COVID-19). <b>Methods:</b> Minimally invasive autopsies from lung, heart, kidney, spleen, bone marrow, liver, pancreas, stomach, intestine, thyroid and skin were performed on three patients died of novel coronavirus pneumonia in Chongqing, China. Hematoxylin and eosin staining (HE) and histochemical staining were performed to investigate the pathological changes of indicated organs or tissues. Immunohistochemical staining was conducted to evaluate the infiltration of immune cells as well as the expression of 2019-nCoV proteins. Real time PCR was carried out to detect the RNA of 2019-nCoV.'))